In [31]:
#!pip install xgboost

In [35]:
#imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import multiprocessing
import gensim
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from langdetect import detect
# from wordcloud import WordCloud, STOPWORDS
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import svm
# from better_profanity import profanity
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.linear_model import Ridge #import ridge 

#imports
%matplotlib inline
import numpy as np 
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest

In [4]:
#import data
X_train = pd.read_pickle('veclyrics_doc2vec_train.pkl')
X_test = pd.read_pickle('veclyrics_doc2vec_test.pkl')
y_train_all = pd.read_pickle('response_doc2vec_train.pkl')
y_test_all = pd.read_pickle('response_doc2vec_test.pkl')
y_train_all = y_train_all.iloc[:,3:10]
y_test_all = y_test_all.iloc[:,3:10]
y_train_one = y_train_all.iloc[:,0]
y_test_one = y_test_all.iloc[:,0]

In [27]:
#doesn't take n_jobs?
#also doesn't want to take multilabel by default
gb = GradientBoostingClassifier(verbose = True)
gb.fit(X_train, y_train_one)

      Iter       Train Loss   Remaining Time 
         1           1.1026           17.09s
         2           1.0875           17.30s
         3           1.0753           17.25s
         4           1.0650           17.21s
         5           1.0563           16.90s
         6           1.0484           16.65s
         7           1.0413           16.48s
         8           1.0359           16.26s
         9           1.0302           16.00s
        10           1.0255           15.87s
        20           0.9930           14.15s
        30           0.9724           12.32s
        40           0.9583           10.52s
        50           0.9480            8.81s
        60           0.9395            6.99s
        70           0.9326            5.21s
        80           0.9260            3.49s
        90           0.9206            1.74s
       100           0.9156            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=True,
                           warm_start=False)

In [28]:
#gb is slow and, for pop at least, way less accurate with default parameters than rf.  
#also, it doesn't easily take multilabel.
gb.score(X_test,y_test_one)

0.7680195599022005

In [6]:
# Gradient boosting classifier
gb = GradientBoostingClassifier(n_estimators=200, random_state=3)
# params_gbc = {
#     "learning_rate": [0.01, 0.05, 0.10, 0.15],
#     "max_depth": [3, 4, 5, 6],
#     "min_samples_leaf": [1, 2, 3, 4, 5],
#     "max_features":['sqrt']
# }

params_gbc = {
    "learning_rate": [0.01, 0.05, 0.10],
    "max_depth": [3, 4, 5],
    "min_samples_leaf": [1, 2, 3],
    "max_features":['sqrt']
}

#reducing from 10 with kfold to cv = 5 to halve fits
gsgbc = GridSearchCV(gb, param_grid=params_gbc, cv=3, scoring="accuracy", n_jobs= -1,verbose = 10)

#gb_best = gsgbc.best_estimator_

gsgbc.fit(X_train,y_train_one)

# Best score
gsgbc.best_score_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   57.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  75 out of  81 | elapsed:  4.1min remaining:   19.6s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  4.3min finished


0.7985231551665118

In [12]:
gsgbc.score(X_test,y_test_one)

0.7699755501222494

In [24]:
rockparam = gsgbc.best_estimator_
rockpred = rockparam.predict(X_test)

In [25]:
#how i would normally check an individual response class
y_pred_rf = rockparam.predict(X_test)
y_pred_train_rf = rockparam.predict(X_train)
print('test report',classification_report(y_test_one, y_pred_rf))
print('train report',classification_report(y_train_one, y_pred_train_rf))

test report               precision    recall  f1-score   support

           0       0.60      0.13      0.21      2459
           1       0.78      0.97      0.87      7766

    accuracy                           0.77     10225
   macro avg       0.69      0.55      0.54     10225
weighted avg       0.74      0.77      0.71     10225

train report               precision    recall  f1-score   support

           0       0.92      0.36      0.52     10190
           1       0.82      0.99      0.90     30708

    accuracy                           0.83     40898
   macro avg       0.87      0.67      0.71     40898
weighted avg       0.85      0.83      0.80     40898



In [18]:
gsgbc.cv_results_

{'mean_fit_time': array([11.3081518 , 13.08590802, 13.12594668, 18.31042401, 19.43094103,
        20.38704999, 27.49654754, 27.80691083, 28.40115023, 15.38440625,
        13.60730775, 11.39607056, 17.16316581, 19.0936594 , 19.81371172,
        26.59359026, 28.10804582, 28.44664598, 16.39201482, 12.78171674,
        10.58074252, 15.82891202, 17.96121391, 19.3528076 , 27.18937818,
        25.85446143, 19.18196519]),
 'std_fit_time': array([1.26626   , 0.27484841, 0.23021418, 0.63289025, 0.45235387,
        0.38011948, 0.87657483, 0.86250159, 1.3267893 , 0.42033844,
        0.75131222, 0.21753072, 0.23068524, 0.82817441, 0.25773188,
        0.39976925, 1.24255933, 2.07529208, 0.48978439, 0.97744221,
        0.21217617, 0.91924774, 0.60534562, 0.33500888, 0.83075881,
        1.36594184, 3.06516744]),
 'mean_score_time': array([0.13264505, 0.12400134, 0.12765892, 0.18417533, 0.15658085,
        0.16954732, 0.21841685, 0.20811049, 0.1863389 , 0.14062421,
        0.12765892, 0.11968017, 0.160

In [42]:
pipe=Pipeline([('select',SelectKBest(k=20)), 
               ('classify', RandomForestClassifier(random_state = 10, max_features = 'sqrt'))])

param_test = {'classify__n_estimators':list(range(20,50,5)), 
              'classify__max_depth':list(range(5,40,5))}
gsearch = GridSearchCV(estimator = pipe, param_grid = param_test, scoring='roc_auc', cv=3, verbose = True, n_jobs = -1)
gsearch.fit(X_test,y_test_one)
print(gsearch.best_params_, gsearch.best_score_)

Fitting 3 folds for each of 42 candidates, totalling 126 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 126 out of 126 | elapsed:   53.7s finished


{'classify__max_depth': 10, 'classify__n_estimators': 45} 0.6881153938106696


In [46]:
#test and train reports appear to be somehow flipped?
#regardless this is not that good
y_pred_gs = gsearch.predict(X_test)
y_pred_train_gs = gsearch.predict(X_train)
print('test report',classification_report(y_test_one, y_pred_gs))
print('train report',classification_report(y_train_one, y_pred_train_gs))

test report               precision    recall  f1-score   support

           0       1.00      0.29      0.45      2459
           1       0.82      1.00      0.90      7766

    accuracy                           0.83     10225
   macro avg       0.91      0.64      0.67     10225
weighted avg       0.86      0.83      0.79     10225

train report               precision    recall  f1-score   support

           0       0.76      0.13      0.23     10190
           1       0.77      0.99      0.87     30708

    accuracy                           0.77     40898
   macro avg       0.77      0.56      0.55     40898
weighted avg       0.77      0.77      0.71     40898



In [ ]:
# #from sklearn.pipeline import make_pipeline
# select = SelectKBest(k = 20)
# clf = RandomForestClassifier(random_state = 10, warm_start = True, 
#                                   n_estimators = 26,
#                                   max_depth = 6, 
#                                   max_features = 'sqrt', verbose = True)
# pipeline = make_pipeline(select, clf)
# pipeline.fit(X_test, y_test_one)

In [ ]:
#veclyrics pkl
#response pkl
#try tfidf

In [25]:
#fitting random forest classifier
#works for multilabel response data by default
rf = RandomForestClassifier(verbose = True, n_jobs= -1)
rf.fit(X_train, y_train_all)

C:\Users\peter\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.9s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.4s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=True,
                       warm_start=False)

In [26]:
#y_pred_rf vs y_test
y_pred_rf = rf.predict(X_test)

acc = []

for i in range(0,7):
    check = y_pred_rf[:,i] == np.array(y_test_all.iloc[:,i])
    accuracy = check.sum()/len(check)
    acc.append(accuracy)

#prints raw accuracy score by genre
print(acc)

#average accuracy score
sum(acc)/len(acc)

#proportion of 1s in each genre column
print(y_test_all.sum()/len(y_test_all))

#why is classification report not matching (anywhere) the manually calculated raw accuracies?
print('test report',classification_report(y_test_all, y_pred_rf))

[0.7467970660146699, 0.8121271393643031, 0.5490464547677262, 0.8497799511002445, 0.7767237163814181, 0.8080195599022005, 0.9282151589242054]
rock                 0.759511
singer-songwriter    0.178289
pop                  0.446455
metal                0.149438
folk                 0.207335
country              0.183081
hip hop / rap        0.072665
dtype: float64
test report               precision    recall  f1-score   support

           0       0.78      0.94      0.85      7766
           1       0.19      0.02      0.03      1823
           2       0.49      0.27      0.35      4565
           3       0.35      0.01      0.01      1528
           4       0.22      0.03      0.05      2120
           5       0.25      0.02      0.04      1872
           6       0.65      0.03      0.05       743

   micro avg       0.69      0.42      0.53     20417
   macro avg       0.42      0.19      0.20     20417
weighted avg       0.52      0.42      0.42     20417
 samples avg       0.70   

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\peter\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.



In [27]:
#appears to only be correct when whole vector of predictions is correct.  our metrics should check genre by genre.
rf.score(X_test,y_test_all)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


0.1710513447432763

In [28]:
#how i would normally check an individual response class
y_pred_rf = rf.predict(X_test)
y_pred_train_rf = rf.predict(X_train)
print('test report',classification_report(y_test_all, y_pred_rf))
print('train report',classification_report(y_train_all, y_pred_train_rf))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\peter\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.



test report               precision    recall  f1-score   support

           0       0.78      0.94      0.85      7766
           1       0.19      0.02      0.03      1823
           2       0.49      0.27      0.35      4565
           3       0.35      0.01      0.01      1528
           4       0.22      0.03      0.05      2120
           5       0.25      0.02      0.04      1872
           6       0.65      0.03      0.05       743

   micro avg       0.69      0.42      0.53     20417
   macro avg       0.42      0.19      0.20     20417
weighted avg       0.52      0.42      0.42     20417
 samples avg       0.70      0.47      0.52     20417

train report               precision    recall  f1-score   support

           0       0.99      1.00      0.99     30708
           1       1.00      0.88      0.93      7281
           2       1.00      0.97      0.98     18372
           3       1.00      0.87      0.93      6221
           4       1.00      0.90      0.94      8429

In [ ]:
#doesn't know n_jobs
#won't take multilabel by default
#doesn't seem to be working

#svc = SVC(probability=True, verbose = True)
#svc.fit(X_train, y_train_one)

In [ ]:
#svc.score(X_test,y_test)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train_one)

In [ ]:
lr.score(X_test,y_test_one)